<a href="https://colab.research.google.com/github/googspell/Notes/blob/main/%D0%90%D1%81%D1%81%D0%B8%D0%BD%D1%85%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

multhidreading -многопоточность подходит для IO bound задач,использует OC,страдает от GIL.Полезно для ускорения выполнения задач или для того что бы текущий поток занялся другой задачей.Любая программа это минимум один процесс и один поток.

Плюсы:
* просто (сравнительно)
* быстро
* не умирает из-за одного

Минусы:
* потребление ресурсов
* неуправляемость
* проблемы потоков

In [ ]:
#threading
import time
import os
import threading
from threading import Thread
import requests
from concurrent.futures import ThreadPoolExecutor



def activity():
  requests.get('https://yandex.ru')
  pid = os.getpid()
  name = threading.current_thread().name
  print(f" Process {pid},name {name}")



def run(threaded=False):
  start = time.time()
  if not threaded:
    for i in range(10):
      activity()


  else:
      #exucutor = ThreadPoolExecutor()
      #for _ in range(10):
        #exucutor.submit(activity)
      #exucutor.shutdown(wait=True)
      # Создаем потоки
      threads =[Thread(target=activity,daemon=True)for _ in range(10)]
      for i in threads:
         # Запускаем потоки
        i.start()
      for i in threads:
         # Ждем завершения потоков
        i.join()

  end = time.time()
  print(f" Time: {end-start} seconds")


if __name__== '__main__':
  run(threaded=True)



 Process 247,name Thread-17 (activity)
 Process 247,name Thread-18 (activity)
 Process 247,name Thread-14 (activity)
 Process 247,name Thread-13 (activity)
 Process 247,name Thread-12 (activity)
 Process 247,name Thread-15 (activity)
 Process 247,name Thread-16 (activity)
 Process 247,name Thread-11 (activity)
 Process 247,name Thread-19 (activity)
 Process 247,name Thread-10 (activity)
 Time: 0.9318909645080566 seconds



multiprocessing -любые задачи iO-bound and CPU -bound.Ускоряет любые задачи,распарелеливая их на ядра процессора(лишь до опредленного момента закон Амдала. Создает несколько процессов,каждый из которых выполняет свою задачу,взаимодействие между ними требует pickle.

Плюсы:
* реальная параллельность любых задач
* не умирает из-за одного
* процессы не зависят друг от друга (у каждого процесса,своя память,GIL)

Минусы:

* потребление ресурсов (памяти,процессора,времени)
* необходимость в сериализации в pickle
* проблемы синхронизации (взаимодействии между процессами)


In [ ]:
#multiprocessing
import time
import os
import multiprocessing
from multiprocessing import Process,Pool
import requests



def activity():
  requests.get('https://yandex.ru')
  pid = os.getpid()
  process_name = multiprocessing.current_process().name
  print(f"Process ID: {pid}, Process Name: {process_name}")



def run(processed=False):
  start = time.time()
  if not processed:
    for i in range(10):
      activity()


  else:
      #exucutor = ThreadPoolExecutor()
      #for _ in range(10):
        #exucutor.submit(activity)
      #exucutor.shutdown(wait=True)
      # Создаем
      processed =[Process(target=activity,daemon=True)for _ in range(10)]
      for i in processed:

        i.start()
      for i in processed:

        i.join()

  end = time.time()
  print(f" Time: {end-start} seconds")


if __name__== '__main__':
  run(processed=True)



Process ID: 5119, Process Name: Process-1
Process ID: 5120, Process Name: Process-2
Process ID: 5122, Process Name: Process-4
Process ID: 5123, Process Name: Process-5Process ID: 5121, Process Name: Process-3Process ID: 5124, Process Name: Process-6
Process ID: 5126, Process Name: Process-8


Process ID: 5127, Process Name: Process-9
Process ID: 5128, Process Name: Process-10Process ID: 5125, Process Name: Process-7

 Time: 0.9071338176727295 seconds


Asyncio ассинхронное выполнение,подходит для IO bound задач,работает ровно 1 поток

Плюсы:

* скорость и экономия времени,вместо x+y+z - max(x,y,z)

* управляемость

* меньше потребление ресурсов

Минусы:

* умирает из-за одного блокирующего вызова

Основные моменты:
* корутина работает как генератор
* async -явный флаг что функция является ассинхронной (корутинной)
* await -явный флан,что в этом месте функция встает на паузу и дает работать другим
* event loop - цикл событий,который отвечает за планирование и запуск корутин














In [ ]:
#Asyncio
import asyncio
import aiohttp
import time

async def async_http():
    async with aiohttp.ClientSession() as session:
        async with session.get('https://yandex.ru') as resp:
            print(f"Status: {resp.status}")

async def main():
    await asyncio.gather(*[async_http() for _ in range(10)])

start = time.time()
await main()  # Используй напрямую await для вызова в jupiternotebook
end = time.time()
print(f"Time: {end - start} seconds")



Status: 400
Status: 400
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Status: 200
Time: 2.520853042602539 seconds


Global Interpreter Lock (GIL)

GIL (Global Interpreter Lock) — это механизм, используемый в интерпретаторе CPython, чтобы обеспечить выполнение только одного потока Python в любой момент времени. Это означает, что даже если у тебя есть несколько потоков, только один из них будет исполнять Python-код в каждый конкретный момент.
Многопоточность (Multithreading)

    Когда мы создаем потоки:

        Мы действительно обращаемся к операционной системе для создания новых потоков. ОС управляет этими потоками через свой планировщик.

        GIL: Внутри каждого процесса Python GIL ограничивает исполнение одного потока Python-кода на процессор в любой момент времени. Однако это ограничение не столь критично для задач ввода-вывода (I/O-bound), потому что потоки могут ждать завершения операций ввода-вывода.

    IO-bound задачи:

        Когда ты выполняешь I/O-bound задачи (например, запросы к базе данных или чтение/запись файлов), потоки часто ждут ответа от внешнего источника (БД, файл и т.д.).

        В это время другие потоки могут выполнять работу, что эффективно использует возможности многопоточности. GIL при этом не является узким местом, так как потоки проводят много времени в ожидании ввода-вывода.

Мультипроцессинг (Multiprocessing)

    Когда мы создаем процессы:

        Мы обращаемся к ОС для создания новых процессов. Каждый процесс имеет свой собственный интерпретатор Python и адресное пространство.

        Обход GIL: В мультипроцессинге каждый процесс имеет свой собственный GIL, что позволяет каждому процессу выполнять Python-код параллельно. Это делает мультипроцессинг эффективным для вычислительно интенсивных задач (CPU-bound).

Асинхронное программирование (Asyncio)

    Асинхронные задачи:

        Асинхронность позволяет выполнять несколько задач в одном потоке, приостанавливая и возобновляя задачи по мере необходимости.

        Цикл событий: asyncio управляет задачами с помощью цикла событий, что позволяет эффективно использовать время ожидания.

Подытожим:

    Многопоточность: Полезна для I/O-bound задач. GIL ограничивает выполнение только одного потока Python-кода на процессор, но это не так критично для задач, ожидающих ввода-вывода.

    Асинхронность (asyncio): Позволяет выполнять несколько задач в одном потоке, эффективно используя время ожидания.

    Мультипроцессинг: Полезен для CPU-bound задач, так как каждый процесс имеет свой собственный GIL и интерпретатор Python.